In [ ]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
hot_100 = pd.read_csv("hot.csv")

In [ ]:
hot_100.drop(columns = ['Unnamed: 0'], inplace = True)
print(hot_100)

In [ ]:
song_random = hot_100.sample()

In [ ]:
songs = []
for i in hot_100['Titles']:
    songs.append(i)

artists = []
for i in hot_100['Artists']:
    artists.append(i)

In [ ]:
def treating_typos_song():
    song_input = input("Please, write the title of the song.")
    if process.extractOne(song_input, songs)[1] > 80:
        song_ = process.extractOne(song_input, songs)[0]
    else:
        song_ = song_input
    return song_

In [ ]:
def treating_typos_artist():
    artist_input = input("Please, write the name of the artist.")
    if process.extractOne(artist_input, artists)[1] > 80:
        artist_ = process.extractOne(artist_input, artists)[0]
    else:
        artist_ = artist_input
    return artist_

In [ ]:
def filter_song_out(song, artist):
    idx = hot_100.index[(hot_100['Song'] == song) & (hot_100['Artist'] == artist)].tolist()[0]
    hot_100.drop(idx)

In [ ]:
spotify_songs = pd.read_csv("musics.csv")
spotify_songs

In [ ]:
X = spotify_songs.drop(columns = spotify_songs[["id", "Unnamed: 0", "Unnamed: 0"]]) #removing the if column so it doesn't get standardized

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
    X
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
X_scaled_df

In [ ]:
#Finding the best number of clusters

from sklearn.cluster import KMeans

K = range(2, 30)
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_scaled_df)
    inertia.append(kmeans.inertia_)

import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
import pickle
from sklearn.metrics import silhouette_score

K = range(2, 30)
silhouette = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(X_scaled_df)

    filename = "Model/kmeans_" + str(k) + ".pickle"
    with open(filename, "wb") as f:
        pickle.dump(kmeans,f)

    silhouette.append(silhouette_score(X_scaled_df, kmeans.predict(X_scaled_df)))


plt.figure(figsize=(16,8))
plt.plot(K, silhouette, 'bx-')
plt.xlabel('k')
plt.ylabel('silhouette score')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Silhouette Method showing the optimal k')

In [ ]:
#Looks like 6 is the optimal number of clusters to work with

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=1234)
kmeans.fit(X_scaled_df)

In [ ]:
clusters = kmeans.predict(X_scaled_df)
pd.Series(clusters).value_counts().sort_index()

In [ ]:
X["cluster"] = clusters

In [ ]:
X

In [ ]:
spotify_df = pd.concat([X, spotify_songs["id"]], axis=1)
spotify_df

In [ ]:
import config

In [ ]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame, display
import random

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= "05734a5b7ac74c91bdda91ff4fd75ba4",
                                                           client_secret= "bccf89cd94604e4f86b2a821003da952"))

In [ ]:
def hot_songs():
    song_ = treating_typos_song()
    artist_ = treating_typos_artist()

    if song_ in songs and artist_ in artists:
        filter_song_out(song_, artist_)
        x = list(song_random["Titles"])[0]
        y = list(song_random["Artists"])[0]
        output_song1 = sp.search(q=y + x, limit=1)
        track_id1=output_song1["tracks"]["items"][0]["id"]
        music1 = IFrame(src="https://open.spotify.com/embed/track/"+track_id1,
                      width="320",
                      height="80",
                     frameborder="0",
                      allowtransparency="true",
                      allow="encrypted-media",
                    )
        print("Your song is in the top 100 and this is our recommendation for you:")
        display(music1)
    else:
        input_song = sp.search(q=artist_ + song_,limit=1)
        track_id=input_song["tracks"]["items"][0]["id"]
        music = IFrame(src="https://open.spotify.com/embed/track/"+track_id,
                      width="320",
                      height="80",
                     frameborder="0",
                      allowtransparency="true",
                      allow="encrypted-media",
                    )


        display(music)
        confirmation = input("Is this the song you chose?(y/n) ")
        print(confirmation)
        if confirmation == "n":
            print("Please, try again.")
        elif confirmation == "y":
            song_df = pd.DataFrame(sp.audio_features(input_song["tracks"]["items"][0]["id"]))
            song_df=song_df[["danceability","energy","loudness","speechiness","acousticness", "instrumentalness","liveness","valence","tempo","id","duration_ms"]]
            song_df2 = song_df.drop(columns = "id")
            song_df2_scaled = scaler.transform(song_df2)
            song_df2_scaled = pd.DataFrame(song_df2_scaled, columns = song_df2.columns)
            cluster = int(kmeans.predict(song_df2_scaled))
            similar_songs = list(np.where(spotify_df["cluster"] == cluster)[0])
            recommendation = random.choice(similar_songs)
            id_recommendation = spotify_df.iloc[recommendation,11]
            output_song = IFrame(src="https://open.spotify.com/embed/track/"+id_recommendation,
                             width="320",
                             height="80",
                             frameborder="0",
                             allowtransparency="true",
                             allow="encrypted-media",
                            )
            print("This is our recommendation for you:")
            return output_song
        else:
            print("You should type y for yes or n for no. Please try again.")

In [102]:
hot_songs()

y
This is our recommendation for you:
